# Holoviews usage

Good resource: [https://holoviews.org/getting_started/Gridded_Datasets.html](https://holoviews.org/getting_started/Gridded_Datasets.html)

Doing n-dimensional plots with traditional plotting libraries can be tedious. `Holoviews` is a library designed for creating multidimensional (especially useful for > 2) plots easily. It is very well suited if you want to explore your data interactively. As we will see, converting `xarray` objects (the most common type of output in postopus) to `holoviews` objects is trivial. The customization level of a `holoviews` plot is very high, because it supports multiple backends, here we will cover `matplotlib` and `bokeh` (the default backend).

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from postopus.octopus_run import Run
from pathlib import Path
%config InlineBackend.figure_formats = ['svg']

input file is already defined in the folder (s. GitLab repo), otherwise we recommend defining it in the notebook

In [ ]:
cd ../../tests/data/benzene/  

Assuming you have octopus in your PATH:

In [ ]:
!octopus > out_gs.log 2>&1

In [ ]:
run = Run(".")

In [ ]:
xa = run.default.scf.density.get_converged(source="cube")  # postopus XArray
xa

## Importing holoviews

In [ ]:
import holoviews as hv
from holoviews import opts  # For setting defaults
hv.extension('bokeh', 'matplotlib')  # Allow for interactive plots

In the following, we are going to convert an `xarray` to a `holoviews Dataset` and then to a `holoviews Image`. Actually, `holoviews Images` are used for 2D plots. Since we have 3D data, it will be converted to a `Holomap` of `Images`. (If we would want to plot for example 1D data, instead of `holoviews Images`, we would use `holoviews Curve`s. The code would be analogous.) This image allows you to slide across time and space and visualize the resulting structure. (If you want to move step-by-step, just select a dimension and use the left and right arrows to navigate). We can use the `kdims` argument to specify which coordinates will serve as the visible axes on the plots (in this example we will choose `x` and `y`). The ones that are left will be controlled by a slider (in this example `t` and `step` will get a slider and a dropdown). When building the image, the default behavior of `holoviews` is to preload the whole data at once with javascript. The loading of the image can be very slow, especially if the amount of data is high. Don't worry, we'll learn how to make it faster!

## Generating a holoviews Dataset

In [ ]:
hv_ds = hv.Dataset(xa)

## Generating a holoviews Image

In [ ]:
hv_im = hv_ds.to(hv.Image, kdims=["x", "y"])

The following two plots will be slowish. 

In [ ]:
hv_im

In [ ]:
type(hv_im)

In [ ]:
hv_im.data # one can see here that each of the samples within the Holomap is an Image

## Customizing plot parameters  with `opts`

In [ ]:
# Sets defaults for all the interactive images in this notebook
# Warning: Currently, there is no way to reset the default settings:
# https://stackoverflow.com/questions/68748393/how-do-i-clear-all-custom-opts-that-ive-set-for-holoviews.
# The only method that works is restarting the kernel, so think about setting anything to default.
opts.defaults(
    opts.Image(cmap='viridis', width=400, height=400), 
)
# <hv object>.opts.clear() / <hv object>.opts(clone=False)  to rollback to default settings if desired,
# from https://holoviews.org/user_guide/Applying_Customizations.html.

In [ ]:
hv_im.opts(opts.Image(title="title"))  # Sets specific opts for a single plot

In [ ]:
# hv.help(hv.Image)  # Lists all the tunable parameters, for the active backend
# hv.help(hv_im)  # Lists the actual configuration of  a specific object

## Time dependent plots

### Methane example

To fasten up the loading process we can make use of the parameter `dynamic=True`. When this parameter is set as `True`, the frames will be loaded one by one, on-demand. (This means that in the background the `Dataset` will be converted into a `DynamicMap` instead of a `Holomap`). Note: if the style of the plotting is not the one that you expected, rerun the cell. The dynamic plots will always take the last used plotting settings, independently of what was declared in the cell. Second note: The dynamic plots cannot be rendered in HTML. If you are viewing this from your webbrowser, you will just see GIFs that should emulate the real plot. If you happen to be in an active notebook session, you can play with them interactively!

In [ ]:
cd ../methane

In [ ]:
!octopus > out_gs.log 2>&1

In [ ]:
run = Run(".")

In [ ]:
xat = run.default.scf.density.get_all(source="vtk")  # XArray Time-dependent

In [ ]:
hv_dst = hv.Dataset(xat)  # convert to holoviews Dataset
hv_imt = hv_dst.to(hv.Image)  # convert to holoviews Image
hv.output(max_frames=10000)  # sets the max number of frames that we can have in a slider-plot

In [ ]:
# hv_imt # Would be very slow! Minutes long!

In [ ]:
hv_dst = hv.Dataset(xat)
hv_imt = hv_dst.to(hv.Image, kdims=["x", "y"], dynamic=True)
hv.output(max_frames=10000)

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt  # fast!

In [ ]:
type(hv_imt)

In [ ]:
hv_imt.data  #  the number of images saved in cache will update if you scroll the plot. The default cache_size is 500

In [ ]:
hv_imt.cache_size

#### Histogram of the data

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt.hist(num_bins=100, log=True).opts(
    opts.Image(colorbar=True, # we set the opts for each hv object independently
              clabel=f"{xat.name} ({xat.units})"), # holoviews doesn't support the labeling of the cmap out of the box  
    opts.Histogram(xlim=(0, 0.5), ylim=(0, 50))
    )  # use base 10 logarithmic samples for the bin edges (width)

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt.hist(num_bins=100).opts(
    opts.Image(colorbar=True,
              clabel=f"{xat.name} ({xat.units})"),
    opts.Histogram(xlim=(0, 0.5), ylim=(0, 50))
    )  # comparison without log

#### Swapping kdims

In [ ]:
hv_imt_step_z = hv_dst.to(hv.Image, kdims=['step', 'z'], dynamic=True)

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt_step_z

### Interference z=0 (slice)

http://holoviews.org/user_guide/Styling_Plots.html
https://holoviews.org/user_guide/Colormaps.html
might come handy for customizing plots

In [ ]:
cd ../interference/

In [ ]:
!octopus > out_td.log 2>&1

In [ ]:
run = Run(".")

In [ ]:
xa = run.Maxwell.td.b_field.x.get_all(source="z=0")

In [ ]:
hv_dst = hv.Dataset(xa)
hv_imt = hv_dst.to(hv.Image, ['x', 'y'])  # needed for generating output files
hv_imt_dynamic = hv_dst.to(hv.Image, ['x', 'y'], dynamic=True)
hv.output(max_frames=3000)

#### Custom plotting options

In [ ]:
# cmap: Which colors should be used
# If cmap is not specified, and dynamic=True, it will change while we scroll. If dynamic=False, it will be fix.
# color_levels: in which ranges should these colors be used
# clim: what is the absolute scale of the colorbar
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt_dynamic.opts(
    colorbar=True, width=500, height=400,
    # cmap=['#0000ff', '#89cff0', '#f75e25', '#ff0000'],
    # color_levels=[-10**-3, -10**-4, 0, 10**-4, 10**-3],
    cmap='seismic',
    clim=(-10**-3, 10**-3),
    clabel=f"{xa.name} ({xa.units})"
)

The middle of the colormap will be in the middle of the min and max values of clim, so e.g. if you want the color map to be symmetric around 0, you should `climmin == -climmax`.

## Saving gifs, htmls, images and videos of plots

### Bokeh for (gifs and) htmls

The recommended output format for interactive plots is `html`. If needed, one can also generate gifs and mp4s with holoviews.

One needs `sudo apt install geckodriver` (or `conda install -c conda-forge geckodriver`) for generating the `bokeh gif`, although one can generate `matplotlib gif`s without it.  
One also needs `sudo apt install ffmpeg` (or `conda install -c conda-forge ffmpeg`)  for generating `matplotlib mp4`s. It is also possible to generate them in RAVEN (s. Advanced section)

The gif and the mp4 generations were commented out because the CI would need geckodriver and ffmpeg. If you have them, just uncomment the code bits!

Note that **we are using hv_imt and NOT hv_imt_dynamic!**

In [ ]:
hv.save(
    hv_imt.opts(
        colorbar=True, width=500, height=400, cmap='seismic', clim=(-10**-3, 10**-3)  # hv.Image args
    ),
    fmt="html", backend="bokeh", filename="test2"  # hv.save args
)

# uncomment if you have `geckodriver`
"""
hv.save(
    hv_imt.opts(
        colorbar=True, width=500, height=400, cmap='seismic', clim=(-10**-3, 10**-3)
    ),
    fmt="gif", backend="bokeh", filename="test3", fps=1
)
"""


hv.save(
    hv_imt.opts(
        colorbar=True, width=500, height=400, cmap='seismic', clim=(-10**-3, 10**-3)
    ),
    fmt="scrubber", backend="bokeh", filename="test20", fps=1
)

Look into the `notebooks` folder, you should see new files in it!

### Matplotlib for gifs and mp4

Bokeh is the default backend when working with holowiews. Although, bokeh doesn't currently support the output of mp4s (Also: the gif generation in bokeh can be slower than with matplotlib). So that the `holoviews` object should be ported to a matploib backend. This shouldn't be complicated. Probably, the only thing one needs to change is`fig_size` instead of `height` and `width`. Recall: `hv.help(hv.Image)` lists all the tunable parameters, for the active backend.

In [ ]:
hv.extension('matplotlib')

For Linux users: VLC player may needed for opening the mp4.

In [ ]:
# uncomment if you have `ffmpeg`
"""
hv.save(
    hv_imt.opts(colorbar=True, fig_inches=10, cmap='seismic', clim=(-10**-3, 10**-3)
               ),
    fmt="mp4", backend="matplotlib", filename="test4", fps=1) 
"""

In [ ]:
hv.save(
    hv_imt.opts(colorbar=True, cmap='seismic', fig_inches=10, clim=(-10**-3, 10**-3)),
    fmt="gif", backend="matplotlib", filename="test5", fps=1
)

In [ ]:
hv.extension('bokeh')  # go back to bokeh as default

### Export individual images

For exporting individual png's the easiest way, is while interacting with a dynamic plot: go to the frame that you are interested in, and click on the save button in the right-hand-bokeh-menu. That's it:

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt_dynamic

In case you need an `svg`, you will need to use matplotlib. Just slice the original xarray at the point where you are interested in:

In [ ]:
xa00 = xa.sel(t=0.0)  # from the plot above look at which step interests you most, let's say t=0.0.
hv_dst00 = hv.Dataset(xa00)
hv_imt00 = hv_dst00.to(hv.Image, ['x', 'y'])

In [ ]:
hv.extension('matplotlib')

For some reason, the following cell causes trouble in the CI pipeline, although when executing it in a notebook there is no problem (problem started end of august). We are going to comment it and maybe address this in the future:

In [ ]:
"""
hv.save(
    hv_imt00.opts(
        colorbar=True, title="step300", fig_size=500, cmap='seismic', clim=(-10**-3, 10**-3), xticks="auto", yticks="auto",
    ),
    fmt="svg", backend="matplotlib", filename="test6", fps=1
)
"""

In [ ]:
hv.help(hv_imt00.opts())

In [ ]:
hv.extension('bokeh', 'matplotlib')  # go back to bokeh as default

## Vectorfields

### Colormap plot

In [ ]:
xav = run.Maxwell.td.b_field.get_all(source="z=0")  # XArray Vector

In [ ]:
hv_dst = hv.Dataset(xav)
hv_imt = hv_dst.to(hv.Image, ['x', 'y'])  # needed for generating outputs
hv_imt_dynamic = hv_dst.to(hv.Image, ['x', 'y'], dynamic=True)
hv.output(max_frames=3000)

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
hv_imt_dynamic.opts(colorbar=True,
                    width=500,
                    height=400,
                    cmap='seismic',
                    clabel=f"{xav.name} ({xav.units})",
                    clim=(-10**-3, 10**-3))

### Arrow plot

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
ds = xr.Dataset(
            {
                "Angle": np.arctan2(
                    xav.sel(x=slice(-10, 10), y=slice(-10, 10), component="y"),
                    xav.sel(x=slice(-10, 10), y=slice(-10, 10), component="x"),
                ),
                "Magnitude": np.sqrt(
                    xav.sel(x=slice(-10, 10), y=slice(-10, 10),component="x") ** 2
                    + xav.sel(x=slice(-10, 10), y=slice(-10, 10), component="y") ** 2
                ),
            }
        ) # polar coordinates required for plotting arrows

In [ ]:
ds

In [ ]:
def _vectorplot(val):
    plot = hv.VectorField(
                data=ds.sel(t=val, method="nearest"),
                kdims=["x", "y"],
                vdims=["Angle", "Magnitude"],
            )
    for dim in plot.kdims: # keep the units
        dim.unit = xav[f"{dim}"].units 
    return plot

In [ ]:
# Note for web users: You should have an active notebook session to interact with the plot
arrow_plot = hv.DynamicMap(_vectorplot, kdims="t").redim.values(t=xav.t.values).opts(color="blue", width=500, height=400) # Zoom-in for seeing the  arrows!
arrow_plot.kdims[0].unit = xav.t.units
arrow_plot

## Advanced 

### Advanced plotting

If the holoviews default plotting does not offer enough customization to your needs, one has two possibilites:
- Use the [holoviews hooks](https://holoviews.org/user_guide/Customizing_Plots.html#Plot-hooks). These hooks will offer the underlying `plot` object from the backend to you. In the case of matplotlib e.g. in the function that you hook to, you could use `fig = plot.handles["fig"]` and `ax = plot.handles["axis"]`. These `fig` and `axes` objects are the ones that you probably know from matplotlib already. You can customize them as you are used to within the function. An example for this can be found in the `case_study_all_final` example, which is referenced under `Application examples`. We recommend exploiting this possibily first, before going to the second one, for still being able of having interactive plots. 
- [Render](https://holoviews.org/user_guide/Plotting_with_Matplotlib.html) the holoviews object into a backend object, e.g. matplotlib, to directly use the classic matplotlib functionalities: `fig = hv.render(hv_object, backend="matplotlib")`, where `ax = fig.axes`. An example for this can be found in the `case_study_all_in_one_exploration` example, which is referenced under `Application examples`.

### Using conda-forge packages on RAVEN.

As said above, for generating `bokeh` `gif`s and `matplotlib` `mp4`s, one needs `geckodriver` and `ffmpeg`, respectively. The latter two are not `Python` packages. Therefore, one cannot install them via `pip`, so they are not installed by default along with `postopus`. Nonetheless, they can be installed via `conda-forge`. The problem is that the standard [MPCDF-RVS-Service](https://docs.mpcdf.mpg.de/doc/visualization/index.html#remote-visualization-and-jupyter-notebook-services) for visualizing notebooks doesn't allow to use custom `anaconda` environments. But there is a rather simple way to overcome this:
1) Login to your raven-account via `ssh` in a terminal  
2) create a conda yaml suited to your needs e.g.:
```yaml
name: holoviews-env
channels:
    - conda-forge
    - defaults
dependencies:
    - python=3.9
    - matplotlib
    - numpy=1.21.5
    - netCDF4
    - pandas
    - prettytable
    - pyvista
    - xarray
    - psutil
    - holoviews
    - datashader
    - selenium
    - pytest
    - Jinja2==3.0.1
    - pip
    - ipykernel
    - geckodriver
    - ffmpeg
    - pip:
       - ase @ git+https://gitlab.com/dremerb/ase.git@xsf_and_cube_merge
       - postopus @ git+https://gitlab.com/octopus-code/postopus.git
```
3)  Create a conda environment with the requirements of the yaml: `conda env create -p /u/user_name/name_of_conda_environment --file=path_to_conf_yml.yml`  
4) `conda deactivate` and `conda activate path_to_conda_env`  
5) `conda install -c conda-forge jupyterlab`  
6) `jupyter-lab`  
7) After a few seconds, a new browser window will pop-up with a jupyter notebook hosted on raven that has all the packages that you need.  


## Final notes

This tutorial is intended to be an introduction to holoviews. We didn't cover everything here. For example, there is another notebook under `/dev` that handles reduction methods in order to plot very dense data. If you need something more sophisticated for large datasets we could think about exploring the dask functionalities, which are also supported by holoviews. Just contact us if you have any questions!